# Comprehensive Vegetable Market Analysis
## Interactive Analytical Notebook

This notebook provides a complete analytical framework for vegetable market intelligence.

**Dataset Overview:**
- annex1.csv: Item catalog (Item Code, Item Name, Category Code, Category Name)
- annex2.csv: Sales transactions (Date, Time, Item Code, Quantity, Unit Price, Sale/Return, Discount)
- annex3.csv: Wholesale prices (Date, Item Code, Wholesale Price)
- annex4.csv: Loss rates (Item Code, Item Name, Loss Rate %)

## 1. Setup and Data Loading

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [ ]:
# Load the master dataset
df = pd.read_csv('outputs/master_dataset.csv')
df['Date'] = pd.to_datetime(df['Date'])

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Date Range: {df['Date'].min()} to {df['Date'].max()}")
print(f"\nFirst few records:")
df.head()

## 2. Data Overview and Quality Check

In [ ]:
# Dataset information
print("Dataset Information:")
print(f"Total Records: {len(df):,}")
print(f"Unique Items: {df['Item Code'].nunique()}")
print(f"Categories: {df['Category Name'].nunique()}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData Types:")
df.dtypes

In [ ]:
# Check for missing values
missing = df.isnull().sum()
if missing.sum() > 0:
    print("Missing Values:")
    print(missing[missing > 0])
else:
    print("✓ No missing values found!")

In [ ]:
# Statistical summary
print("Statistical Summary:")
df.describe()

## 3. Price Analysis

In [ ]:
# Price distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogram
axes[0, 0].hist(df['Wholesale Price (RMB/kg)'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Wholesale Price (RMB/kg)', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Price Distribution', fontsize=14, fontweight='bold')
axes[0, 0].grid(axis='y', alpha=0.3)

# Box plot
axes[0, 1].boxplot(df['Wholesale Price (RMB/kg)'].dropna(), vert=True)
axes[0, 1].set_ylabel('Wholesale Price (RMB/kg)', fontsize=12)
axes[0, 1].set_title('Price Box Plot', fontsize=14, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)

# Log scale histogram
axes[1, 0].hist(np.log1p(df['Wholesale Price (RMB/kg)']), bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1, 0].set_xlabel('Log(Price + 1)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('Log-Transformed Price Distribution', fontsize=14, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# QQ plot for normality
stats.probplot(df['Wholesale Price (RMB/kg)'].dropna(), dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot (Normality Test)', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate statistics
print(f"\nPrice Statistics:")
print(f"Mean: ¥{df['Wholesale Price (RMB/kg)'].mean():.2f}/kg")
print(f"Median: ¥{df['Wholesale Price (RMB/kg)'].median():.2f}/kg")
print(f"Std Dev: ¥{df['Wholesale Price (RMB/kg)'].std():.2f}/kg")
print(f"CV: {df['Wholesale Price (RMB/kg)'].std() / df['Wholesale Price (RMB/kg)'].mean():.2f}")

In [ ]:
# Top items by price
top_items = df.groupby('Item Name')['Wholesale Price (RMB/kg)'].mean().nlargest(15)

plt.figure(figsize=(14, 8))
top_items.plot(kind='barh', color='darkgreen', edgecolor='black')
plt.xlabel('Average Wholesale Price (RMB/kg)', fontsize=12)
plt.ylabel('Item Name', fontsize=12)
plt.title('Top 15 Most Expensive Items', fontsize=16, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nTop 15 Items by Price:")
print(top_items)

## 4. Loss Rate Analysis

In [ ]:
# Loss rate distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(df['Loss Rate (%)'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0].set_xlabel('Loss Rate (%)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Loss Rate Distribution', fontsize=14, fontweight='bold')
axes[0].axvline(df['Loss Rate (%)'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["Loss Rate (%)"].mean():.2f}%')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Box plot by category
df.boxplot(column='Loss Rate (%)', by='Category Name', ax=axes[1])
axes[1].set_xlabel('Category', fontsize=12)
axes[1].set_ylabel('Loss Rate (%)', fontsize=12)
axes[1].set_title('Loss Rate by Category', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
plt.suptitle('')

plt.tight_layout()
plt.show()

print(f"\nLoss Rate Statistics:")
print(f"Mean: {df['Loss Rate (%)'].mean():.2f}%")
print(f"Median: {df['Loss Rate (%)'].median():.2f}%")
print(f"High Loss Items (>15%): {len(df[df['Loss Rate (%)'] > 15]['Item Name'].unique())}")

In [ ]:
# Items with highest loss rates
high_loss = df.groupby('Item Name')['Loss Rate (%)'].mean().nlargest(15)

plt.figure(figsize=(14, 8))
high_loss.plot(kind='barh', color='crimson', edgecolor='black')
plt.xlabel('Average Loss Rate (%)', fontsize=12)
plt.ylabel('Item Name', fontsize=12)
plt.title('Top 15 Items with Highest Loss Rates', fontsize=16, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nTop 15 Items by Loss Rate:")
print(high_loss)

## 5. Category Analysis

In [ ]:
# Category performance
category_stats = df.groupby('Category Name').agg({
    'Wholesale Price (RMB/kg)': ['mean', 'std', 'min', 'max'],
    'Loss Rate (%)': 'mean',
    'Item Code': 'count'
}).round(2)

category_stats.columns = ['Avg Price', 'Std Price', 'Min Price', 'Max Price', 'Avg Loss %', 'Count']
category_stats = category_stats.sort_values('Avg Price', ascending=False)

print("Category Performance Summary:")
print(category_stats)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

category_stats['Avg Price'].plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='black')
axes[0].set_xlabel('Category', fontsize=12)
axes[0].set_ylabel('Average Price (RMB/kg)', fontsize=12)
axes[0].set_title('Average Price by Category', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

category_stats['Avg Loss %'].plot(kind='bar', ax=axes[1], color='coral', edgecolor='black')
axes[1].set_xlabel('Category', fontsize=12)
axes[1].set_ylabel('Average Loss Rate (%)', fontsize=12)
axes[1].set_title('Average Loss Rate by Category', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Time Series Analysis

In [ ]:
# Monthly price trends
monthly_avg = df.groupby(df['Date'].dt.to_period('M')).agg({
    'Wholesale Price (RMB/kg)': ['mean', 'std', 'min', 'max', 'count']
})
monthly_avg.columns = ['Mean', 'Std', 'Min', 'Max', 'Count']
monthly_avg.index = monthly_avg.index.to_timestamp()

plt.figure(figsize=(16, 8))
plt.plot(monthly_avg.index, monthly_avg['Mean'], marker='o', linewidth=2, label='Average Price', markersize=6)
plt.fill_between(monthly_avg.index,
                monthly_avg['Mean'] - monthly_avg['Std'],
                monthly_avg['Mean'] + monthly_avg['Std'],
                alpha=0.3, label='±1 Std Dev')
plt.plot(monthly_avg.index, monthly_avg['Max'], linestyle='--', alpha=0.5, label='Max Price')
plt.plot(monthly_avg.index, monthly_avg['Min'], linestyle='--', alpha=0.5, label='Min Price')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Wholesale Price (RMB/kg)', fontsize=12)
plt.title('Monthly Price Trends with Variability', fontsize=16, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nMonthly Statistics:")
print(monthly_avg.head(10))

In [ ]:
# Seasonal patterns
seasonal = df.groupby(df['Date'].dt.month)['Wholesale Price (RMB/kg)'].mean()

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.figure(figsize=(14, 6))
plt.bar(range(1, 13), seasonal.values, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average Price (RMB/kg)', fontsize=12)
plt.title('Seasonal Price Patterns', fontsize=16, fontweight='bold')
plt.xticks(range(1, 13), months)
plt.grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(seasonal.values, 1):
    plt.text(i, v + 0.1, f'{v:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\nSeasonal Average Prices:")
for i, price in enumerate(seasonal.values, 1):
    print(f"{months[i-1]}: ¥{price:.2f}/kg")

## 7. Correlation Analysis

In [ ]:
# Correlation matrix
corr_cols = ['Wholesale Price (RMB/kg)', 'Loss Rate (%)', 'Effective Price (RMB/kg)', 'Month', 'DayOfWeek']
correlation = df[corr_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt='.3f', cmap='coolwarm',
           center=0, square=True, linewidths=2, cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelation Matrix:")
print(correlation)

In [ ]:
# Price vs Loss Rate scatter plot
plt.figure(figsize=(14, 8))

for category in df['Category Name'].unique():
    cat_data = df[df['Category Name'] == category].sample(min(200, len(df[df['Category Name'] == category])))
    plt.scatter(cat_data['Loss Rate (%)'],
               cat_data['Wholesale Price (RMB/kg)'],
               alpha=0.6, s=50, label=category)

plt.xlabel('Loss Rate (%)', fontsize=12)
plt.ylabel('Wholesale Price (RMB/kg)', fontsize=12)
plt.title('Price vs Loss Rate by Category', fontsize=16, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate correlation
corr, p_value = stats.pearsonr(df['Loss Rate (%)'].dropna(), 
                                df.loc[df['Loss Rate (%)'].dropna().index, 'Wholesale Price (RMB/kg)'].dropna())
print(f"\nCorrelation between Price and Loss Rate: {corr:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Significance: {'Significant' if p_value < 0.05 else 'Not significant'} at α=0.05")

## 8. Volatility Analysis

In [ ]:
# Calculate price volatility (coefficient of variation) for each item
volatility = df.groupby('Item Name').agg({
    'Wholesale Price (RMB/kg)': ['mean', 'std', 'count']
})
volatility.columns = ['Mean Price', 'Std Price', 'Count']
volatility['CV'] = volatility['Std Price'] / volatility['Mean Price']
volatility = volatility[volatility['Count'] >= 10]  # Filter items with enough data
volatility = volatility.sort_values('CV', ascending=False)

print("Top 20 Most Volatile Items:")
print(volatility.head(20))

# Visualize
plt.figure(figsize=(14, 10))
volatility['CV'].head(20).plot(kind='barh', color='orange', edgecolor='black')
plt.xlabel('Coefficient of Variation', fontsize=12)
plt.ylabel('Item Name', fontsize=12)
plt.title('Top 20 Most Volatile Items (Price Variability)', fontsize=16, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Anomaly Detection

In [ ]:
# Detect outliers using IQR method
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (series < lower_bound) | (series > upper_bound)

price_outliers = detect_outliers_iqr(df['Wholesale Price (RMB/kg)'])
loss_outliers = detect_outliers_iqr(df['Loss Rate (%)'])

print(f"Price Outliers: {price_outliers.sum()} ({price_outliers.sum()/len(df)*100:.2f}%)")
print(f"Loss Rate Outliers: {loss_outliers.sum()} ({loss_outliers.sum()/len(df)*100:.2f}%)")

# Visualize outliers
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Price outliers
axes[0].scatter(df.index, df['Wholesale Price (RMB/kg)'], c=price_outliers, cmap='RdYlGn_r', alpha=0.6, s=10)
axes[0].set_xlabel('Record Index', fontsize=12)
axes[0].set_ylabel('Wholesale Price (RMB/kg)', fontsize=12)
axes[0].set_title('Price Outlier Detection (Red = Outlier)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Loss rate outliers
axes[1].scatter(df.index, df['Loss Rate (%)'], c=loss_outliers, cmap='RdYlGn_r', alpha=0.6, s=10)
axes[1].set_xlabel('Record Index', fontsize=12)
axes[1].set_ylabel('Loss Rate (%)', fontsize=12)
axes[1].set_title('Loss Rate Outlier Detection (Red = Outlier)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Business Insights & Recommendations

In [ ]:
# Generate key insights
print("="*80)
print("KEY BUSINESS INSIGHTS")
print("="*80)

# Price trends
first_month = df.groupby(df['Date'].dt.to_period('M'))['Wholesale Price (RMB/kg)'].mean().iloc[0]
last_month = df.groupby(df['Date'].dt.to_period('M'))['Wholesale Price (RMB/kg)'].mean().iloc[-1]
price_change = (last_month - first_month) / first_month * 100
print(f"\n1. PRICE TRENDS")
print(f"   Overall price change: {price_change:+.2f}%")
print(f"   Price volatility (CV): {df['Wholesale Price (RMB/kg)'].std() / df['Wholesale Price (RMB/kg)'].mean():.2f}")

# Category insights
print(f"\n2. CATEGORY PERFORMANCE")
cat_stats = df.groupby('Category Name').agg({
    'Wholesale Price (RMB/kg)': 'mean',
    'Loss Rate (%)': 'mean'
}).round(2)
highest_price = cat_stats['Wholesale Price (RMB/kg)'].idxmax()
highest_loss = cat_stats['Loss Rate (%)'].idxmax()
print(f"   Highest priced category: {highest_price} (¥{cat_stats.loc[highest_price, 'Wholesale Price (RMB/kg)']:.2f}/kg)")
print(f"   Highest loss category: {highest_loss} ({cat_stats.loc[highest_loss, 'Loss Rate (%)']:.2f}%)")

# Loss rate insights
print(f"\n3. LOSS RATE ANALYSIS")
print(f"   Average loss rate: {df['Loss Rate (%)'].mean():.2f}%")
high_loss_count = len(df[df['Loss Rate (%)'] > 15]['Item Name'].unique())
print(f"   Items with >15% loss rate: {high_loss_count}")
print(f"   Potential savings from 5% loss reduction: ¥{df['Wholesale Price (RMB/kg)'].sum() * 0.05:,.2f}")

# Seasonal insights
print(f"\n4. SEASONAL PATTERNS")
seasonal = df.groupby(df['Date'].dt.month)['Wholesale Price (RMB/kg)'].mean()
peak_month = seasonal.idxmax()
low_month = seasonal.idxmin()
variation = (seasonal.max() - seasonal.min()) / seasonal.mean() * 100
print(f"   Peak price month: {months[peak_month-1]} (¥{seasonal[peak_month]:.2f}/kg)")
print(f"   Lowest price month: {months[low_month-1]} (¥{seasonal[low_month]:.2f}/kg)")
print(f"   Seasonal variation: {variation:.1f}%")

print("\n" + "="*80)
print("STRATEGIC RECOMMENDATIONS")
print("="*80)
print("\n1. HIGH PRIORITY: Supply Chain Optimization")
print(f"   - Target {high_loss_count} items with loss rates >15%")
print("   - Expected impact: 5-10% margin improvement")
print("   - Implementation: Cold chain enhancement, staff training")

print("\n2. MEDIUM PRIORITY: Dynamic Pricing Strategy")
print("   - Implement for volatile categories")
print("   - Expected impact: Better margin protection")
print("   - Implementation: Real-time pricing algorithms")

print("\n3. MEDIUM PRIORITY: Seasonal Inventory Management")
print(f"   - Optimize for {variation:.1f}% seasonal variation")
print("   - Expected impact: Reduced carrying costs")
print("   - Implementation: Predictive inventory models")

## 11. Custom Analysis Section

Use this section for your own exploratory analysis and custom visualizations.

In [ ]:
# Your custom analysis here
# Example: Analyze a specific item or category

# item_name = "Your Item Name Here"
# item_data = df[df['Item Name'] == item_name]
# ... your analysis

## Summary

This notebook has provided a comprehensive analysis of the vegetable market data, including:

1. **Data Quality**: Complete dataset with no missing values
2. **Price Analysis**: Identified pricing trends, top items, and volatility
3. **Loss Analysis**: Analyzed loss rates and identified high-risk items
4. **Category Performance**: Compared categories across multiple dimensions
5. **Time Series**: Discovered seasonal patterns and trends
6. **Correlations**: Identified relationships between variables
7. **Anomalies**: Detected outliers for further investigation
8. **Strategic Insights**: Generated actionable recommendations

**Next Steps:**
- Review the Streamlit dashboard for interactive exploration
- Run the main analytics framework for updated reports
- Implement recommended strategies and monitor results